<a href="https://colab.research.google.com/github/ekomey/SVR-Regression/blob/Training/Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler,RobustScaler

from google.colab import files
uploaded = files.upload()

data = pd.read_csv("Metro_Interstate_Traffic_Volume.csv")
data.head()

le =LabelEncoder()

Holiday_labels = le.fit_transform(data['holiday'])

{index: label for index , label in enumerate(le.classes_)}

weather_main = le.fit_transform(data['weather_main'])
{index: label for index , label in enumerate(le.classes_)}

weather_description = le.fit_transform(data['weather_description'])
{index: label for index , label in enumerate(le.classes_)}

data['holiday'] = le.fit_transform(data['holiday'])
data['weather_main'] = le.fit_transform(data['weather_main'])
data['weather_description'] = le.fit_transform(data['weather_description'])


# aggregate rows with same date-time
data_agg = data.groupby('date_time',as_index=False).agg('max')

data_agg['date_time'] = pd.to_datetime(data_agg['date_time'])
# convert date_time column to datetime format

# create column with hours from date_time
data_agg['hour'] = data_agg['date_time'].dt.hour
data_agg['hour'].value_counts().plot(kind='bar')

# create column with day of the week from date_time
data_agg['day_of_week'] = data_agg['date_time'].dt.dayofweek
data_agg['day_of_week'].value_counts().plot(kind='bar')

# create column with month from date_time
data_agg['month'] = data_agg['date_time'].dt.month
data_agg['month'].value_counts().plot(kind='bar')

data_agg.head()
data_agg.drop(['date_time'], axis=1)

# save preprocessed data to csv
data_agg.to_csv("Metro-Interstate-Traffic-Volume-Encoded.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import tensorflow.compat.v1 as tf
from sklearn.model_selection import train_test_split


# We have imported all dependencied
import io
df = pd.read_csv('Metro-Interstate-Traffic-Volume-Encoded.csv')
#del data['date_time']
df = df.drop(['date_time'],axis=1) # Drop Date feature
print(df)

print(df.info()) # Overview of dataset

df = df.dropna(inplace=False)  # Remove all nan entries.

df_train = df[:32460]    # 60% training data and 40% testing data
df_test = df[32460:]
# scaler = MinMaxScaler() # For normalizing dataset

# # We want to predict Close value of stock 
# X_train = scaler.fit_transform(df_train.drop(['traffic_volume'],axis=1).values())
# y_train = scaler.fit_transform(df_train['traffic_volume'].values())
# # y is output and x is features.
# X_test = scaler.fit_transform(df_test.drop(['traffic_volume'],axis=1).values())
# y_test = scaler.fit_transform(df_test['traffic_volume'].values())

def normalization(raw_data):
    for col_num in range(raw_data.shape[1]):
        if raw_data.iloc[:,col_num].dtype == np.float or raw_data.iloc[:,col_num].dtype == np.int:
            raw_data.iloc[:,col_num] = (raw_data.iloc[:,col_num] - raw_data.iloc[:,col_num].mean()) / (raw_data.iloc[:,col_num].max() - raw_data.iloc[:,col_num].min())
    return raw_data

dataset = normalization(df)
x_data = df.drop('traffic_volume',axis=1)
y_data = df['traffic_volume'][:,None]
X_train,X_test,Y_train,Y_test=train_test_split(x_data,y_data,test_size=0.2,random_state=0)

Y_train = np.reshape(Y_train, (-1, 1))
y_test = np.reshape(Y_test, (-1, 1))
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
print(scaler_x.fit(X_train))
xtrain_scale=scaler_x.transform(X_train)
print(scaler_x.fit(X_test))
xval_scale=scaler_x.transform(X_test)
print(scaler_y.fit(Y_train))
ytrain_scale=scaler_y.transform(Y_train)
print(scaler_y.fit(Y_test))
yval_scale=scaler_y.transform(Y_test)

def denormalize(df,norm_data):
    df = df['traffic_volume'].values.reshape(-1,1)
    norm_data = norm_data.reshape(-1,1)
    scl = MinMaxScaler()
    a = scl.fit_transform(df)
    new = scl.inverse_transform(norm_data)
    return new

"""
Above written function for denormalizatio of data after normalizing
this function will give original scale of values.
In normalization we step down the value of data in dataset.
"""

def neural_net_model(X_data,input_dim):
    W_1 = tf.Variable(tf.random_uniform([input_dim,10]))
    b_1 = tf.Variable(tf.zeros([10]))
    layer_1 = tf.add(tf.matmul(X_data,W_1), b_1)
    layer_1 = tf.nn.relu(layer_1)

    # layer 1 multiplying and adding bias then activation function
    W_2 = tf.Variable(tf.random_uniform([10,10]))
    b_2 = tf.Variable(tf.zeros([10]))
    layer_2 = tf.add(tf.matmul(layer_1,W_2), b_2)
    layer_2 = tf.nn.relu(layer_2)
    # layer 2 multiplying and adding bias then activation function
    W_O = tf.Variable(tf.random_uniform([10,1]))
    b_O = tf.Variable(tf.zeros([1]))
    output = tf.add(tf.matmul(layer_2,W_O), b_O)
    # O/p layer multiplying and adding bias then activation function
    # notice output layer has one node only since performing #regression
    return output
"""
neural_net_model is function applying 2 hidden layer feed forward neural net.
Weights and biases are abberviated as W_1,W_2 and b_1, b_2 
These are variables with will be updated during training.
"""
tf.disable_v2_behavior()

xs = tf.placeholder("float")
ys = tf.placeholder("float")
output = neural_net_model(xs,10)
cost = tf.reduce_mean(tf.square(output-ys))
# our mean squared error cost function

train = tf.train.GradientDescentOptimizer(0.001).minimize(cost)
# Gradinent Descent optimiztion just discussed above for updating weights and biases

c_t = []
c_test = []

with tf.Session() as sess:
    # Initiate session and initialize all vaiables
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    for i in range(30):
        for j in range(X_train.shape[0]):
            sess.run([cost,train],feed_dict= {xs:X_train.iloc[j,:].values.reshape(1,10), ys:Y_train[j]})
            # Run cost and train with each sample
        c_t.append(sess.run(cost, feed_dict={xs:X_train,ys:Y_train}))
        c_test.append(sess.run(cost, feed_dict={xs:X_test,ys:Y_test}))
        print('Epoch :',i,'Cost :',c_t[i])
    pred = sess.run(output, feed_dict={xs:X_test})
    # predict output of test data after training
    print('Cost :',sess.run(cost, feed_dict={xs:X_test,ys:Y_test}))
    Y_test = denormalize(df_test,Y_test)
    pred = denormalize(df_test,pred)
    #Denormalize data     
    plt.plot(range(Y_test.shape[0]),Y_test,label="Original Data")
    plt.plot(range(Y_test.shape[0]),pred,label="Predicted Data")
    plt.legend(loc='best')
    plt.ylabel('Traffic Volume Value')
    plt.xlabel('Days')
    plt.title('Prediction vs Actual')
    plt.show()
    # if input('Save model ? [Y/N]') == 'Y':
    #     saver.save(sess,'yahoo_dataset.ckpt')
    #     print('Model Saved')

In [ ]:
#PLOTTING
plt.figure(figsize=(15,8))
plt.scatter (Y_test, pred) 
range = [Y_test.min (), pred.max ()] 
plt.plot (range, range, 'red') 
plt.title('Predicted vs Actual ')
plt.xlabel ('Actual') 
plt.ylabel ('Predicted ') 
plt.show ()

plt.figure(figsize=(15,8))
plt.plot(Y_test,label="Original Data")
plt.plot(pred,label="Predicted Data")
plt.legend(loc='best')
plt.ylabel('Traffic Volume Value')
plt.xlabel('Days')
plt.title('Prediction vs Actual')
plt.show()